# Text Analytics

NLP(Natural Language Processing)는 문어와 구어를 다루는 AI(인공 지능)의 한 분야입니다. NLP를 사용하여 텍스트나 음성에서 의미를 추출하거나 의미 있는 답변을 자연어로 조합하는 솔루션을 구축할 수 있습니다.

Microsoft Azure *Cognitive Services*에는 텍스트에서 핵심 문구를 식별하고 감정을 바탕으로 텍스트를 분류하는 등 몇 가지 기본 NLP 기능을 제공하는 *Text Analytics* 서비스가 포함되어 있습니다.

![노트북을 읽고 있는 로봇](./images/NLP.jpg)

예를 들어 *Margie's Travel*이라는 가상의 단체가 고객에게 호텔 숙박에 대한 리뷰를 제출하도록 권장한다고 가정해 보세요. Text Analytics 서비스를 사용하면 핵심 문구를 추출하여 리뷰를 요약하거나, 긍정적 리뷰와 부정적 리뷰를 판단하거나, 리뷰 텍스트를 분석하여 위치나 사람과 같은 알려진 엔티티에 대한 언급을 확인할 수 있습니다.

## 리뷰 문서 검토

먼저 고객들이 남긴 몇 가지 호텔 리뷰를 살펴보겠습니다.

리뷰는 텍스트 파일로 되어 있습니다. 리뷰를 보려면 셀 왼쪽에 있는 **셀 실행**(&#9655;) 단추를 클릭하여 아래의 코드를 실행하면 됩니다.

In [2]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

review3.txt
Good location and helpful staff, but on a busy road.
The Lombard Hotel, San Francisco, USA
8/16/2018
We stayed here in August after reading reviews. We were very pleased with location, just behind Chestnut Street, a cosmopolitan and trendy area with plenty of restaurants to choose from. The
Marina district was lovely to wander through, very interesting houses. Make sure to walk to the San Francisco Museum of Fine Arts and the Marina to get a good view of Golden Gate bridge and the city. On a bus route and easy to get into centre. Rooms were clean with plenty of room and staff were friendly and helpful. The only down side was the noise from Lombard Street so ask to have a room furthest away from traffic noise.

review2.txt
Tired hotel with poor service
The Royal Hotel, London, United Kingdom
5/6/2018
This is a old hotel (has been around since 1950's) and the room furnishings are average - becoming a bit old now and require changing. The internet didn't work and had to come t

## Cognitive Services 리소스 만들기

이 리뷰에서 텍스트를 분석하기 위해 **Text Analytics** Cognitive Service를 사용할 수 있습니다. 이 기능을 사용하려면 Azure 구독에서 **Text Analytics** 또는 **Cognitive Services** 리소스를 프로비전해야 합니다. (사용하려는 유일한 서비스이거나 사용량을 따로 추적하려는 경우 Text Analytics 리소스를 합니다. 아니면 Cognitive Services 리소스를 사용하여 Text Analytics 서비스를 다른 Cognitive Services와 결합함으로써 개발자가 단일 엔드포인트와 키를 사용하여 액세스하도록 할 수 있습니다.)

아직 없다면 다음 단계를 따라 Azure 구독에서 **Cognitive Services** 리소스를 만듭니다.

> **참고**: 이미 Cognitive Services 리소스를 보유하고 있다면 Azure Portal에서 **빠른 시작** 페이지를 열고 키 및 엔드포인트를 아래의 셀로 복사하기만 하면 됩니다. 리소스가 없다면 아래의 단계를 따라 리소스를 만듭니다.

1. 다른 브라우저 탭에서 Azure Portal(https://portal.azure.com) 을 열고 Microsoft 계정으로 로그인합니다.
2. **&#65291;리소스 만들기** 단추를 클릭하고, *Cognitive Services*를 검색하고, 다음 설정을 사용하여 **Cognitive Services** 리소스를 만듭니다.
    - **구독**: *사용자의 Azure 구독*.
    - **리소스 그룹**: *고유한 이름의 새 리소스 그룹을 선택하거나 만듭니다*.
    - **지역**: *사용 가능한 지역을 선택합니다*.
    - **이름**: *고유한 이름을 입력합니다*.
    - **가격 책정 계층**: S0
    - **알림을 읽고 이해했음을 확인합니다**. 선택됨.
3. 배포가 완료될 때까지 기다립니다. 그런 다음에 Cognitive Services 리소스로 이동하고, **개요** 페이지에서 링크를 클릭하여 서비스의 키를 관리합니다. 클라이언트 애플리케이션에서 Cognitive Services 리소스에 연결하려면 엔드포인트 및 키가 필요합니다.

### Cognitive Services 리소스의 키 및 엔드포인트 가져오기

Cognitive Services 리소스를 사용하려면 클라이언트 애플리케이션에 해당 엔드포인트 및 인증 키가 필요합니다.

1. Azure Portal에 있는 Cognitive Service 리소스의 **키 및 엔드포인트** 페이지에서 리소스의 **Key1**을 복사하고 아래 코드에 붙여 넣어 **YOUR_COG_KEY**를 대체합니다.
2. 리소스의 **엔드포인트**를 복사하고 아래 코드에 붙여 넣어 **YOUR_COG_ENDPOINT**를 대체합니다.
3. 녹색 <span style="color:green">&#9655;</span> 단추를 클릭하여 아래의 셀에 있는 코드를 실행합니다.

In [3]:
cog_key = '149b9a076c7d48dd98f306cec2e540c4'
cog_endpoint = 'https://ai-900-kwcgs.cognitiveservices.azure.com/'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Ready to use cognitive services at https://ai-900-kwcgs.cognitiveservices.azure.com/ using key 149b9a076c7d48dd98f306cec2e540c4


## 언어 감지
먼저 이 리뷰가 작성된 언어를 식별하겠습니다.

In [4]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

review3.txt
 - Language: English
 - Code: en
 - Score: 1.0

review2.txt
 - Language: English
 - Code: en
 - Score: 1.0

review1.txt
 - Language: English
 - Code: en
 - Score: 1.0

review4.txt
 - Language: English
 - Code: en
 - Score: 1.0



## 핵심 문구 추출

이제 고객 리뷰의 텍스트를 분석하여 주요 화제에 대해 언급하는 핵심 문구를 식별할 수 있습니다.

In [7]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    print(key_phrase_analysis.documents[review_num])
     # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

review3.txt

Key Phrases:
{'additional_properties': {}, 'id': 'review3.txt', 'key_phrases': ['helpful staff', 'Lombard Street', 'Good location', 'Chestnut Street', 'Lombard Hotel', 'Marina district', 'traffic noise', 'San Francisco Museum of Fine Arts', 'good view of Golden Gate bridge', 'trendy area', 'USA', 'city', 'bus route', 'busy road', 'centre', 'restaurants', 'Rooms', 'interesting houses', 'reviews'], 'statistics': None}
	 helpful staff
	 Lombard Street
	 Good location
	 Chestnut Street
	 Lombard Hotel
	 Marina district
	 traffic noise
	 San Francisco Museum of Fine Arts
	 good view of Golden Gate bridge
	 trendy area
	 USA
	 city
	 bus route
	 busy road
	 centre
	 restaurants
	 Rooms
	 interesting houses
	 reviews


review2.txt

Key Phrases:
{'additional_properties': {}, 'id': 'review2.txt', 'key_phrases': ['old hotel', 'Royal Hotel', 'Tired hotel', 'London', 'United Kingdom', 'room furnishings', 'poor service', 'British Museum', 'website', 'office rooms', 'flight home', 'inte

핵심 문구는 각 리뷰에서 가장 중요한 화제를 이해하는 데 도움이 될 수 있습니다. 예를 들어 "도움이 되는 직원" 또는 "서비스 불량"이라는 문구가 포함된 리뷰는 리뷰어의 주 관심사를 어느 정도 나타낼 수 있습니다.

## 감정 판단

*감정 점수*를 바탕으로 리뷰를 *긍정적* 또는 *부정적*으로 분류하는 것이 유용할 수 있습니다. 이것 역시 Text Analytics 서비스를 사용하여 수행할 수 있습니다.

In [9]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each revieww
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score
    print(sentiment_score)

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({:.10f})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

0.9999995231628418
review3.txt : positive (0.9999995232)
5.662441253662109e-07
review2.txt : negative (0.0000005662)
0.9999973773956299
review1.txt : positive (0.9999973774)
2.0623207092285156e-05
review4.txt : negative (0.0000206232)


## 알려진 엔터티 추출

*엔터티*는 텍스트에서 언급되면서 몇몇 잘 알려진 유형의 항목을 나타내는 요소입니다. 위치, 사람, 날짜를 예로 들 수 있습니다. 리뷰에 언급된 날짜와 장소에 관심이 있다고 가정해 보세요. 다음 코드를 사용하여 날짜와 장소를 찾을 수 있습니다.

In [12]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # 값을 확인 print("entity :: {}".format(entity))
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location', 'Organization']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

review3.txt
 - Location: Lombardy (https://en.wikipedia.org/wiki/Lombardy)
 - Organization: Lombard Hotel 
 - Location: San Francisco (https://en.wikipedia.org/wiki/San_Francisco)
 - Location: San Francisco 
 - Location: USA 
 - DateTime: 8/16/2018 
 - DateTime: August 
 - Location: Chestnut Street (Philadelphia) (https://en.wikipedia.org/wiki/Chestnut_Street_(Philadelphia))
 - Location: Marina 
 - Location: Marina District, San Francisco (https://en.wikipedia.org/wiki/Marina_District,_San_Francisco)
 - Organization: San Francisco Museum of Fine Arts 
 - Organization: Museum of Fine Arts, Boston (https://en.wikipedia.org/wiki/Museum_of_Fine_Arts,_Boston)
 - Location: Golden Gate Bridge (https://en.wikipedia.org/wiki/Golden_Gate_Bridge)
 - Location: Golden Gate 
 - Location: Lombard Street (San Francisco) (https://en.wikipedia.org/wiki/Lombard_Street_(San_Francisco))
review2.txt
 - Location: The Royal Hotel (https://en.wikipedia.org/wiki/The_Royal_Hotel)
 - Location: London (https://en.

일부 엔터티는 연관된 Wikipedia 페이지가 있을 정도로 잘 알려져 있으며, 이 경우 Text Analytics 서비스는 해당 페이지의 URL을 반환합니다.

## 자세한 내용

Text Analytics 서비스에 대한 자세한 내용은 [Text Analytics 서비스 설명서](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)를 참조하세요.